In [ ]:
%matplotlib inline
import os
from pprint import pprint
import matplotlib.pyplot as plt
import sciunit
import sciunit.scores
import neuronunit
from neuronunit.models.reduced import ReducedModel
from neuronunit.capabilities import ProducesSpikes

In [ ]:
# Assumes imported neuronunit is from source, e.g. pip install -e
path = os.path.join(neuronunit.__path__[0],'models/NeuroML2/LEMS_2007One.xml')
# Instantiate three identical models
models = [ReducedModel(path, name='Izhikevich', backend='jNeuroML') for i in range(2)]

In [ ]:
# Change the parameters of the second model
models[1].set_attrs({'izhikevich2007Cell':{'a':'0.04 per_ms'}})
models[1].name = 'Izhikevich_new'

In [ ]:
for model in models:
    plt.plot(model.get_membrane_potential(),label=model)
plt.legend();

In [ ]:
# A new M2M test which will compare the equality of spike counts across models
class MyTest(sciunit.Test):
    required_capabilities = (ProducesSpikes,)
    score_type = sciunit.scores.ZScore
    def generate_prediction(self,model):
        count = model.get_spike_count()
        return count

In [ ]:
test = MyTest(observation={'mean':8, 'std':3, 'url':'http://somewhere.thathasdata.com'})

In [ ]:
s = test.judge(models)

In [ ]:
# The score matrix
s

In [ ]:
x = test.json(string=False)
pprint(x)

In [ ]:
def check_url(x):
    if not x.json(string=False).get('url'):
        print("Model has no associated url; please set the url attribute of the model")
check_url(models[0])

In [ ]:
# How do you want your GitHub URLs?  
lems_url = ("https://github.com/scidash/neuronunit/blob/master/neuronunit/"
            "models/NeuroML2/LEMS_2007One.xml")
models[0]._url = lems_url
models[1]._url = lems_url

In [ ]:
check_url(models[0])

In [ ]:
# Has fewer attributes because most of them came from the LEMS file so those are assumed
# to still be the values in memory.  If we need to extract these then I need to find a
# way to read them from the file; however, there may be no general way to do so for all
# LEMS files, and certainly not for all possible models.  
x = models[0].json(string=False)
pprint(x)

In [ ]:
# Has more attributes because some of the original attributes from the LEMS file 
# were replaced with new ones in this session
x = models[1].json(string=False)
pprint(x)

In [ ]:
# Select one score from the score matrix
score = s['Izhikevich','MyTest']
x = score.json(string=False)
pprint(x)

In [ ]:
# Add additional properties to the JSON output
x = score.json(add_props=True,string=False)
pprint(x)

In [ ]:
# Nicer formatting for the output
x = score.json(add_props=True,string=False)
pprint(x)

In [ ]:
test2 = MyTest(observation={'mean':7, 'std':3.8, 'url':'http://somewhereelse.thathasdata.com'},name='Second Test')
suite = sciunit.TestSuite([test,test2],name='My Suite')
x = suite.json(add_props=True,string=False)
pprint(x)

In [ ]:
score_matrix = suite.judge(models)

In [ ]:
x = score_matrix.json(add_props=True,string=False)
pprint(x)